<h1 style="text-align:center;">Computational Intelligence</h1>
<h2 style="text-align:center;">Classification with MLP</h2>
<h4 style="text-align:center;">by H. Naderan</h4>
<h5 style="text-align:center;">
Mechanical Engineering Department<br>
Amirkabir University of Technology
</h5>

***

### Importing libraries

In [153]:
import numpy as np
import os
os.environ["KERAS_BACKEND"] = "torch"
import keras_core as keras
import pandas as pd

### Loading data

In [154]:
trainData = pd.read_csv("mnist_train.zip")
testData = pd.read_csv("mnist_test.zip")
Xtrain, Ytrain = trainData.iloc[:, 1:].to_numpy(), trainData.iloc[:, 0].to_numpy()
Xtest, Ytest = testData.iloc[:, 1:].to_numpy(), testData.iloc[:, 0].to_numpy()

### Visualize data samples

In [ ]:
plt.figure(figsize=(18, 5))
for i in range(3):
    plt.subplot(1, 3, i + 1)
    plt.axis(True)
    plt.imshow(np.reshape(Xtrain[i], (28, 28)), cmap='gray')
    plt.subplots_adjust(wspace=0.2, hspace=0.2)

### Preprocess data

#### Scaling input features

In [126]:
Xtrain = Xtrain.astype("float32")/255
Xtest = Xtest.astype("float32")/255

#### One-hot encoding labels

In [127]:
Ytrain = keras.utils.to_categorical(Ytrain)
Ytest  = keras.utils.to_categorical(Ytest)

In [ ]:
Ytrain[1]

### Building the model

In [ ]:
model = keras.Sequential([
    keras.layers.Input(shape=(Xtrain.shape[1],)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation="softmax")
])
model.summary()

In [139]:
model.compile(
    loss=keras.losses.CategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.CategoricalAccuracy(name="acc"),
    ],
)

### Model training

In [140]:
batch_size = 128
epochs = 20

In [ ]:
trainState = model.fit(
    Xtrain,
    Ytrain,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.15,
)

### Plotting training state

In [147]:
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter)

In [148]:
def plot_results(metrics, title=None, ylabel=None, ylim=None, metric_name=None, color=None):

    fig, ax = plt.subplots(figsize=(15, 4))

    if not (isinstance(metric_name, list) or isinstance(metric_name, tuple)):
        metrics = [metrics,]
        metric_name = [metric_name,]

    for idx, metric in enumerate(metrics):    
        ax.plot(metric, color=color[idx])

    plt.xlabel("Epoch")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.xlim([0, 20])
    plt.ylim(ylim)
    # Tailor x-axis tick marks
    ax.xaxis.set_major_locator(MultipleLocator(5))
    ax.xaxis.set_major_formatter(FormatStrFormatter('%d'))
    ax.xaxis.set_minor_locator(MultipleLocator(1))
    plt.grid(True)
    plt.legend(metric_name)   
    plt.show()
    plt.close()

In [ ]:
trainLoss = trainState.history["loss"]
trainAcc  = trainState.history["acc"]
validLoss = trainState.history["val_loss"]
validAcc  = trainState.history["val_acc"]

plot_results([trainLoss, validLoss],        
            ylabel="Loss", 
            ylim = [0.0, 0.5],
            metric_name=["Training Loss", "Validation Loss"],
            color=["g", "b"]);

plot_results([trainAcc, validAcc], 
            ylabel="Accuracy",
            ylim = [0.9, 1.0],
            metric_name=["Training Accuracy", "Validation Accuracy"],
            color=["g", "b"])

### Making predictions

In [ ]:
Ypred = model.predict(Xtest)

In [ ]:
print(Ytest[0])
print(Ypred[0])